In [1]:
import windc_data as wd

In [2]:
datasets = {
    "bea_gsp":wd.BeaGsp,
    "bea_pce":wd.BeaPce,
    "bea_supply_det":wd.BeaSupplyDet,
    "bea_supply" : wd.BeaSupply,
    "bea_use_det" : wd.BeaUseDet,
    "bea_use" : wd.BeaUse,
    "census_sgf" : wd.CensusSGF,
    "cfs" : wd.CFS,
    "eia_crude_price" : wd.EiaCrude,
    "eia_emissions" : wd.EiaEmissions,
    "eia_heatrate" : wd.EiaHeatrate,
    "eia_seds" : wd.EiaSeds,
    "nass" : wd.Nass,
    "state_exim" : wd.StateExIm,
    "emission_rate":wd.EmissionRate
    }


In [4]:
data_dir = r"C:\Users\mphillipson\Documents\WiNDC\windc_raw_data\windc_3_0_0"

w1 = wd.WindcEnvironment(data_dir,
                         load = datasets,
                         verbose = False)

#w1.to_gdx(r"C:\Users\mphillipson\Documents\WiNDC")

w1.to_gdx(r"C:\Users\mphillipson\Documents\WiNDC\buildstream_julia\base_data")

In [10]:
w1.keys()

dict_keys(['r', 'sr', 'i', 'fd', 'ts', 'va', 'i_det', 'sector_map', 'seds_src', 'yr', 'version', 'supply_units', 'use_units'])

In [21]:
w1["fd"]

,windc_label,description_code
83,pce,Personal consumption expenditures (F010)
84,equipment,Nonresidential private fixed investment in equ...
85,intelprop,Nonresidential private fixed investment in int...
86,residential,Residential private fixed investment (F02R)
87,structures,Nonresidential private fixed investment in str...
88,changinv,Change in private inventories (F030)
90,defense,National defense: Consumption expenditures (F06C)
91,def_equipment,Federal national defense: Gross investment in ...
92,def_intelprop,Federal national defense: Gross investment in ...
93,def_structures,Federal national defense: Gross investment in ...


In [15]:
w1["i"]

,windc_label,description_code
0,agr,Farms (111CA)
1,fof,"Forestry, fishing, and related activities (113FF)"
2,oil,Oil and gas extraction (211)
3,min,"Mining, except oil and gas (212)"
4,smn,Support activities for mining (213)
...,...,...
68,fen,Federal government enterprises (GFE)
69,slg,State and local general government (GSLG)
70,sle,State and local government enterprises (GSLE)
71,oth,Noncomparable imports and rest-of-the-world ad...


In [4]:
import gamstransfer as gt
import pandas as pd

In [17]:
X = gt.Container(r"C:\Users\mphillipson\Documents\WiNDC\windc_build\core\windc_base.gdx")

--- Warning: The GAMS version (41.1.0) differs from the API version (41.0.0).
--- Warning: The GAMS version (41.1.0) differs from the API version (41.0.0).


In [22]:
X["cfsdata_ma_units"].records

,uni_0,uni_1,uni_2,uni_3,uni_4,value
0,104,104,212,10,millions of us dollars (USD),2.204431
1,104,104,212,11,millions of us dollars (USD),3.736687
2,104,104,212,19,millions of us dollars (USD),0.351043
3,104,104,212,31,millions of us dollars (USD),0.542113
4,104,104,212,34,millions of us dollars (USD),3.618401
...,...,...,...,...,...,...
287204,99999,99999,331,40,millions of us dollars (USD),1341.953532
287205,99999,99999,331,41,millions of us dollars (USD),311.817305
287206,99999,99999,331,43,millions of us dollars (USD),6.737957
287207,99999,99999,331,13,millions of us dollars (USD),0.149248


In [ ]:
gdx_container = gt.Container()

old_gdx_sets = w1._gdx_sets()

sets_to_keep = ["yr"]

gdx_sets = {e:old_gdx_sets[e] for e in sets_to_keep}



for key,d in gdx_sets.items():

    if key not in gdx_container.data:
        num_domain = 1
        if type(d['elements']) == type(pd.DataFrame()):
            num_domain = len(d['elements'].columns)-1
        gdx_container.addSet(key,["*"]*num_domain,records = d['elements'],description = d['text'])


gdx_container.write("test.gdx")


In [ ]:
old_gdx_sets.keys()

In [ ]:
gdx_container["yr"].records

In [ ]:
#gdx_container["r"]

gdx_container.write("test.gdx")

In [ ]:
for key in gdx_container.data:
    print(gdx_container[key].hasDuplicateRecords())
    print(gdx_container[key].hasDomainViolations())


In [ ]:
gdx_container["i"].records

In [ ]:
gdx_container["i_det"].records.isnull()

In [ ]:
gdx_container.data.keys()

In [ ]:
def to_gdx(self,output_dir,output_name = "windc.gdx"):
    import gamstransfer as gt
    
    gdx_container = gt.Container()
    
    gdx_sets = self._gdx_sets()
    
    for key,d in gdx_sets.items():
    #    for key,d in [(k,d) for (k,d) in gdx_dict.items() if d['type'] == 'set']:
        if key not in gdx_container.data:
            num_domain = 1
            if type(d['elements']) == type(pd.DataFrame()):
                num_domain = len(d['elements'].columns)-1
            gdx_container.addSet(key,["*"]*num_domain,records = d['elements'],description = d['text'])
    
    
    for key in self.data:
        self.data[key].build_gdx(gdx_container)
        
    gdx_container.write(os.path.join(output_dir,output_name))

In [ ]:
years = (1997,2018)

pd.DataFrame(zip(list(range(years[0],years[1]+1)),list(range(years[0],years[1]+1))),columns = ['yr',"description"]),